In [3]:
# example of loading the vgg16 model
from keras.applications.vgg16 import VGG16
# load model
vgg16_model = VGG16()
# summarize the model
vgg16_model.summary()

Using TensorFlow backend.






553467904/553467096 [==============================] - 6s 0us/step






Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________

In [4]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
type(vgg16_model)

keras.engine.training.Model

In [6]:
train_path = 'gdrive/My Drive/ML project/TrafficSigns/Train'
test_path = 'gdrive/My Drive/ML project/TrafficSigns/Test'

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_batches = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=62,
    class_mode='categorical',
    subset='training') # set as training data

valid_batches = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size=(224, 224),
    batch_size=62,
    class_mode='categorical',
    subset='validation') # set as validation data

#test_batches = train_datagen.flow_from_directory(
#    test_path,
 #  target_size=(224, 224),
 #   batch_size=64,
  #  class_mode='categorical')

Found 31372 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [7]:
vgg16_model.layers.pop()

In [8]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
model = Sequential()
for layer in vgg16_model.layers:
  model.add(layer)

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
for layer in model.layers:
  layer.trainable = False

In [0]:
model.add(Dense(43, activation='softmax'))

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [14]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit_generator(train_batches, steps_per_epoch=32, validation_data=valid_batches, validation_steps=32, epochs=10, verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
 - 2072s - loss: 3.6194 - acc: 0.0650 - val_loss: 3.4800 - val_acc: 0.0635
Epoch 2/10
 - 2039s - loss: 3.4561 - acc: 0.0902 - val_loss: 3.4013 - val_acc: 0.0932
Epoch 3/10
 - 2037s - loss: 3.3814 - acc: 0.1028 - val_loss: 3.3562 - val_acc: 0.0524
Epoch 4/10
 - 2020s - loss: 3.3232 - acc: 0.1089 - val_loss: 3.2705 - val_acc: 0.1112
Epoch 5/10
 - 2031s - loss: 3.2842 - acc: 0.1255 - val_loss: 3.3060 - val_acc: 0.1275
Epoch 6/10
 - 2032s - loss: 3.2519 - acc: 0.1336 - val_loss: 3.2101 - val_acc: 0.1598
Epoch 7/10
 - 2031s - loss: 3.2146 - acc: 0.1472 - val_loss: 3.1883 - val_acc: 0.1562
Epoch 8/10
 - 2016s - loss: 3.1750 - acc: 0.1542 - val_loss: 3.1601 - val_acc: 0.1686
Epoch 9/10
 - 2036s - loss: 3.1642 - acc: 0.1517 - val_loss: 3.1817 - val_acc: 0.1588
Epoch 10/10
 - 2034s - loss: 3.1247 - acc: 0.1774 - val_loss: 3.1253 - val_acc: 0.1694


In [1]:
test_batches = train_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=62,
    class_mode='categorical')

NameError: ignored

In [0]:
predictions = model.predict_generator(test_batches, steps=32, verbose=0)

In [0]:
loss, acc = model.evaluate_generator(test_batches, steps=32, verbose=0)

In [19]:
print(loss)
print(acc)

3.0539448261260986
0.19354838132858276
